In [ ]:
import httplib2
from apiclient import errors
from apiclient.discovery import build
from oauth2client.client import OAuth2WebServerFlow
import os
from dotenv import load_dotenv
project_folder = os.path.expanduser('/Users/oleksandrdagayev/code/jupyter_SEO/')  # adjust as appropriate
load_dotenv(os.path.join(project_folder, '.env')) #create a text file called .env and paste  
import numpy as np
import pandas as pd
import json
from collections import defaultdict as dd

In [ ]:
# Redirect URI for installed apps
REDIRECT_URI = 'urn:ietf:wg:oauth:2.0:oob'

CLIENT_ID = os.getenv('CLIENT_ID') #get this from setting up a Google App see here 
CLIENT_SECRET = os.getenv('CLIENT_SECRET')

OAUTH_SCOPE = 'https://www.googleapis.com/auth/webmasters.readonly'

# Run through the OAuth flow and retrieve credentials
flow = OAuth2WebServerFlow(CLIENT_ID, CLIENT_SECRET, OAUTH_SCOPE, REDIRECT_URI)
authorize_url = flow.step1_get_authorize_url()
print('Go to the following link in your browser: ' + authorize_url) #this will generate a link - make sure to click the link

In [ ]:
#the above gets you to the credential setp.

In [ ]:
code = "" #this was my authcode used with my instance - you will need to use your own
credentials = flow.step2_exchange(code)

# Create an httplib2.Http object and authorize it with our credentials
http = httplib2.Http()
http = credentials.authorize(http)

In [ ]:
webmaster_search_analytics = build('webmasters', 'v3', http=http) #build the service

In [ ]:
targetdomain = '' #specify your domain here. Make sure to include protocol and if domain is www. or not. 

In [ ]:
request = {
    'startDate': '2020-01-11', #all requests must include a start AND end date
    'endDate': '2020-01-12',
    'dimensions': ['date','query','page'], #select the dimensions of your response
    #'dimensionsFilterGroups':[{'filters'}:[{'dimension':'device','expression':'mobile'}]] - this is sample filter
    'rowLimit':25000, #this is the maximum row limit for GSC export data
    'startRow':0
      }

In [ ]:
test_response = webmaster_search_analytics.searchanalytics().query(siteUrl=targetdomain,body=request).execute()

In [ ]:
test_response

In [ ]:
test_data = {'date':[],'searchQuery':[],'page':[], 'clicks':[],'impressions':[],'ctr':[], 'position':[]}
    
for i in test_response['rows']:
    date = i['keys'][0] #assumes that date is first
    test_data['date'].append(date)
    searchQuery = i['keys'][1] #assumes that query is second
    test_data['searchQuery'].append(searchQuery)
    page = i['keys'][2] #assumes that query is second
    test_data['page'].append(page)
    clicks = i['clicks']
    test_data['clicks'].append(clicks)
    impressions = i['impressions']
    test_data['impressions'].append(impressions)
    ctr = i['ctr']
    test_data['ctr'].append(ctr)
    position = i['position']
    test_data['position'].append(position)

In [ ]:
pd.DataFrame(test_data) #this is meant to validate that everything you're doing is correct so far

In [ ]:
def get_all_response(domain,request): #this is a function that agreggates responses for API calls over 25000 rows
    data = {'date':[],'searchQuery':[],'page':[], 'clicks':[],'impressions':[],'ctr':[], 'position':[]}
    startRow = 0
    request['startRow'] = startRow
    initresponse = webmaster_search_analytics.searchanalytics().query(siteUrl=targetdomain,body=request).execute()
    last_call_len = len(initresponse['rows'])
    mapped_dictionary = map_response_to_dict(initresponse, data)
    while last_call_len == 25000:
        print('this is running')
        startRow = startRow + 25000
        request['startRow'] = startRow
        response = webmaster_search_analytics.searchanalytics().query(siteUrl=targetdomain,body=request).execute()
        last_call_len = len(response['rows'])
        mapped_dictionary = map_response_to_dict(response, data)
    return mapped_dictionary
        

In [ ]:
def map_response_to_dict(response, data): # this turns the JSON response into an easy to manipulate dictionary for dataframes
    for i in response['rows']:
        date = i['keys'][0] #assumes that date is first
        data['date'].append(date)
        searchQuery = i['keys'][1] #assumes that query is second
        data['searchQuery'].append(searchQuery)
        page = i['keys'][2] #assumes that page is third
        data['page'].append(page)
        clicks = i['clicks']
        data['clicks'].append(clicks)
        impressions = i['impressions']
        data['impressions'].append(impressions)
        ctr = i['ctr']
        data['ctr'].append(ctr)
        position = i['position']
        data['position'].append(position)
    return data

In [ ]:
request1 = {
    'startDate': '2020-01-11', #all requests must include a start AND end date
    'endDate': '2020-01-12',
    'dimensions': ['date','query','page'], #select the dimensions of your response
    #'dimensionsFilterGroups':[{'filters'}:[{'dimension':'device','expression':'mobile'}]] - this is sample filter
    'rowLimit':25000, #this is the maximum row limit for GSC export data
    'startRow':0
      }

In [ ]:
request2 = {
    'startDate': '2019-01-11', #all requests must include a start AND end date
    'endDate': '2019-01-12',
    'dimensions': ['date','query','page'], #select the dimensions of your response
    #'dimensionsFilterGroups':[{'filters'}:[{'dimension':'device','expression':'mobile'}]] - this is sample filter
    'rowLimit':25000, #this is the maximum row limit for GSC export data
    'startRow':0
      }

In [ ]:
big_dict1 = get_all_response(targetdomain,request1) #pushing all data to a dictionary

In [ ]:
seoDataFrame1 = pd.DataFrame(big_dict1) #turning the dictionary into a PandaDataFrame

In [ ]:
seoDataFrame1 #this is our first set of data for the first date range

In [ ]:
big_dict2 = get_all_response(targetdomain,request2)

In [ ]:
seoDataFrame2 = pd.DataFrame(big_dict2) #this is our second set of data for the first date range

In [ ]:
seoDataFrame2

In [ ]:
summedSearchQueryDateRange1 = seoDataFrame1.groupby(
    ['searchQuery'] # this is set to be aggregated only on searchQuery right now. We can change this to be ['searchQuery','page'] to be on a searchQuery & page level
).agg({
    'clicks':"sum",
    'impressions':"sum",
    'ctr':"mean",
    'position':"mean"
}).sort_values('clicks',ascending=False) #we're defining how we're aggregating the information here

In [ ]:
summedSearchQueryDateRange1 #taking a look at our aggregated data

In [ ]:
summedSearchQueryDateRange2 = seoDataFrame2.groupby(
    ['searchQuery']
).agg({
    'clicks':"sum",
    'impressions':"sum",
    'ctr':"mean",
    'position':"mean"
}).sort_values('clicks',ascending=False) 
#this is the same code as for the above except now we're aggregating the 2nd data frame

In [ ]:
summedSearchQueryDateRange2 #checking out our data frame

In [ ]:
mergedSummedSEODataFrames = pd.merge(summedSearchQueryDateRange2, summedSearchQueryDateRange1, on=['searchQuery'], how='inner')

#this is the merged version of the two data frames
#please note that this implementation uses an inner join which means that comparison is only done on search queries that are present in both data frames. 
#To change this try using "left" or "right" as the parameter on how, depending on which dataset you want to be using

In [ ]:
mergedSummedSEODataFrames

In [ ]:
#below is where the difference is calculated for two date ranges

In [ ]:
mergedSummedSEODataFrames['clicks_diff'] = mergedSummedSEODataFrames['clicks_x'] - mergedSummedSEODataFrames['clicks_y']

In [ ]:
mergedSummedSEODataFrames['impressions_diff'] = mergedSummedSEODataFrames['impressions_x'] - mergedSummedSEODataFrames['impressions_y']

In [ ]:
mergedSummedSEODataFrames['ctr_diff'] = mergedSummedSEODataFrames['ctr_x'] - mergedSummedSEODataFrames['ctr_y']

In [ ]:
mergedSummedSEODataFrames['position_diff'] = mergedSummedSEODataFrames['position_y'] - mergedSummedSEODataFrames['position_x']

In [ ]:
mergedSummedSEODataFrames

In [ ]:
#the below exports everything to a .csv file. Feel free to define your own export path here

In [ ]:
mergedSummedSEODataFrames.to_csv('/Users/oleksandrdagayev/code/jupyter_SEO/sample.csv')